In [1]:
import networkx as nx
#from tqdm import tqdm
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time



In [2]:
print('Loading graph...')

G_Whole =  nx.read_adjlist('../../train.txt', delimiter='\t', create_using=nx.DiGraph(), nodetype=int)


Loading graph...


In [ ]:
print('Loading partial graph...')

#edges_subgraph = pd.read_csv('train_17MB.txt', header=None, names=['source','target','class'])
#edges = edges_subgraph[edges_subgraph['class'] ==1]
#edges_graph = edges[['source','target']]
#edges_graph.to_csv('edges_graph.csv', header=False, index=None)
G_limited =  nx.read_edgelist('edges_graph.csv',delimiter=',',create_using=nx.DiGraph(), nodetype=int)


In [3]:
# Setting the graph (change to G_limited for the crawled subgraph)
G = G_Whole

In [ ]:
''' No longer in use (slow Pandas implementation)
dataset = pd.read_csv('dataset.csv')

features = np.array_split(dataset, 30)

print('Splitting dataset into 30.')
for i in tqdm(range(28,30)):
    print('Dataset', i, '. number of samples:', len(features[i]))

    features[i]['pref_attachment'] = features[i].apply(lambda row: pref_attach(row.source, row.target), axis=1)
    features[i]['total_friends'] = features[i].apply(lambda row: total_friends(row.source, row.target), axis=1)
    features[i]['common_friends'] = features[i].apply(lambda row: common_friends(row.source, row.target), axis=1)
    features[i]['jacard_coef'] = features[i].apply(lambda row: jacard_coef(row.source, row.target), axis=1)
    features[i].to_csv('data_features_%s.csv' % (i))
'''


# Importing Sampled Data

In [118]:
file_path = './../Data/sample.txt' # change to desired sampled dataset (remove header name because it's a string)



In [121]:
# if you have a class column 
train_data = np.loadtxt(file_path,  usecols=range(3))
train_int = train_data.astype(np.int64)



In [ ]:
# If your file does not have class column, create one (assuming it's all existing edges, i.e. class 1)


train_data = np.loadtxt(file_path, delimiter=',', usecols=range(2))

classes = np.repeat(1, len(train_int))
classes = classes.reshape(-1,1)
train_int = np.append(train_int, classes, axis=1)
train_int = train_int.astype(np.int64)
#np.savetxt(file_path, train_int)


# Computing Features

In [110]:
'''
Pasting features from file for faster Graph access. 

Feature function parameters are modified for a vectorised implementation.

'''

%reload_ext autoreload
%autoreload 2
# Importing the features from Features2 file
#from features2 import common_friends, jacard_coef, total_friends, trans_friends, opposite_friends, pref_attach


# File for testing feature implementation
import networkx as nx
import time
import pandas as pd
#G =  nx.read_adjlist('/Users/williamrudd/documents/MSc/COMP90051/train.txt', delimiter='\t',create_using=nx.DiGraph() )

# Defining intersection and union of lists.
#G.degree('20388')
# Trying to be consistent with notation from the paper to avoid confusion.
# Neighborhood functions
def Gamma(u):
    return set(G.successors(u)).union(G.predecessors(u))
def Gamma_in(u):
    return G.predecessors(u)
def Gamma_out(u):
    return G.successors(u)
def Gamma_plus(u):
    return set(Gamma(u)).union({u})


## maybe not use.
def Gamma_union(u):
    return set(Gamma(u)).union(Gamma(u[1]))

def Gamma_union_plus(u):
    return set(Gamma_plus(u)).union(Gamma_plus(u[1]))
# DEFINING FEATURES


# FASTEST edge features => total of 4 features.
def total_friends(x):
    return len(set(Gamma(x[0])).union(Gamma(x[1])))
# PREFERENTIAL ATTACHMENT
def pref_attach(u):
    return len(Gamma(u[0]))*len(Gamma(u[1]))

# COMMON FRIENDS
def common_friends(u):
    return len(set(Gamma(u[0])).intersection(Gamma(u[1])))

# JACARD'S COEFFICIENT
def jacard_coef(u):
    return common_friends(u)/total_friends(u)

# TRANSITIVE FRIENDS
def trans_friends(u):
    return len(set(G.successors(u[0])).intersection(G.predecessors(u[1])))

# OPPOSITE FRIENDS
def opposite_friends(u):
    return int(G.has_edge(u[1],u[0]))

# Vertex features are all very fast and already implemented in networkx
# example:
#G.degree('20388')
#G.in_degree('20388')
#G.out_degree('20388')


# SLOW FEATURES - need to be rewritten in a more efficient manner. To start, I will
# be taking a look at

def friends_measure(u):
    counter = 0
    for i in Gamma(u[0]):
        for j in Gamma(u[1]):
            if (i == j) or (G.has_edge(i,j) == True) or (G.has_edge(j,i) == True):
                counter = counter + 1
    return counter


# compute edge features

# SUBGRAPH FUNCTIONS/FEATURES
# I represent these in subgraph form rather than edge list, as specified in the
# paper.

# nh subgraph for vertex (unused as of this moment)
def nh_subgraph_vertex(u):
    return G.subgraph(Gamma(u[0]))

def nh_subgraph_vertex_plus(u):
    return G.subgraph(Gamma_plus(u[0]))

# nh subgraph for edge
def nh_subgraph_edge(u):
    return G.subgraph(set(Gamma(u[0])).union(Gamma(u[1])))

def nh_subgraph_edge_plus(u):
    return G.subgraph(set(Gamma_plus(u[0])).union(Gamma_plus(u[1])))

def inner_subgraph(u):
    g1 = Gamma(u[0])
    g2 = Gamma(u[1])
    # some function that grabs a full edgelist
    # for some edge (a,b)
    g = nh_subgraph_edge(u[0])
    e = g.edges()
    inner = nx.DiGraph() # create empty subgraph structure
    for i in e:
        if ((i[0] in g1) and (i[1] in g2)) or ((i[1] in g1) and (i[0] in g2)):
            inner.add_edge(i[0],i[1]) # add this edge to inner
    return inner

# scc nh subgraph
def scc_nh(u):
    return nx.number_strongly_connected_components(nh_subgraph_edge(u))

# scc nh subgraph +
def scc_nh_plus(u):
    return nx.number_strongly_connected_components(nh_subgraph_edge_plus(u))

# scc inner subgraph
def scc_inner(u):
    return nx.number_strongly_connected_components(inner_subgraph(u))


# Related: number of nodes in the above subgraphs
def size_nh(u):
    return nx.number_of_nodes(nh_subgraph_edge(u))

# size nh subgraph +
def size_nh_plus(u):
    return nx.number_of_nodes(nh_subgraph_edge_plus(u))

# size inner subgraph
def size_inner(u):
    return nx.number_of_nodes(inner_subgraph(u))



### Friendship Features

In [123]:

def features():
    t0 = time.time()
    common_friend = np.apply_along_axis(common_friends, 1, train_int)
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))


    t0 = time.time()
    total_friend = np.apply_along_axis(total_friends, 1, train_int)
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))
    
    
    t0 = time.time()
    jacard_coefs = common_friend / total_friend
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))


    t0 = time.time()
    trans_friend = np.apply_along_axis(trans_friends, 1, train_int)
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))


    t0 = time.time()
    opposite_friend = np.apply_along_axis(opposite_friends, 1, train_int)
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))


    t0 = time.time()
    prefs = np.apply_along_axis(pref_attach, 1, train_int)
    t1 = time.time()
    total = t1-t0
    print(total, total/len(train_int))
    
    return common_friend, jacard_coefs, total_friend, trans_friend, opposite_friend, prefs



In [ ]:
common_friend, jacard_coefs, total_friend, trans_friend, opposite_friend, prefs = features()

## Degree Features

In [ ]:

# Compute degree features
def degree_features():
    t = time.time()
    degree_sources = np.apply_along_axis(G.degree, 0,train_int[:,:1])[1]
    t1 = time.time() - t
    print(t1)

    t = time.time()
    degree_source_in = np.apply_along_axis(G.in_degree,0,train_int[:,:1])[1]
    t2 = time.time() - t
    print(t2)

    t = time.time()
    degree_source_out = np.apply_along_axis( G.out_degree ,0,train_int[:,:1])[1]
    t3 = time.time() - t
    print(t3)

    t = time.time()
    degree_target = np.apply_along_axis(G.degree, 0,train_int[:,1:2])[1]
    t4 = time.time() - t
    print(t4)

    t = time.time()
    degree_target_in = np.apply_along_axis(G.in_degree,0,train_int[:,1:2])[1]
    t5 = time.time() - t
    print(t5)

    t = time.time()
    degree_target_out = np.apply_along_axis(G.out_degree,0,train_int[:,1:2])[1]
    t6 = time.time() - t
    print(t6)
    
    return degree_sources, degree_source_in, degree_source_out, degree_target, degree_target_in, degree_target_out

In [ ]:
degree_sources, degree_source_in, degree_source_out, degree_target, degree_target_in, degree_target_out = degree_features()

# Saving Features

### Saving Complete Dataset

In [ ]:
#change this path name everytime you compute for a new dataset
file_path = 'crawled_4_seeds'


column_names = ['pref_attach','total_friends','common_friends','degree_source','degree_source_in','degree_source_out','degree_target','degree_target_in','degree_target_out','class']

computed_features = pd.DataFrame(columns=column_names)
computed_features['pref_attach'] = prefs
computed_features['total_friends'] = total_friend
computed_features['common_friends'] = common_friend
computed_features['degree_source'] = degree_sources
computed_features['degree_source_in'] = degree_source_in
computed_features['degree_source_out'] = degree_source_out
computed_features['degree_target'] = degree_target
computed_features['degree_target_in'] = degree_target_in
computed_features['degree_target_out'] = degree_target_out
computed_features['class'] = train_int[:,2:3]

computed_features.to_csv(str(file_path,'/train.csv'), index=None)

### Saving Individual Features (not in use) 

In [ ]:
G = G_limited
file_path = 'crawled1_limited'
features()
np.savetxt(str(file_path,'/opposite_friends.txt'), opposite_friend)
np.savetxt(str(file_path,'/friends.txt'), total_friend)
np.savetxt(str(file_path,'/transistive_friends.txt'), trans_friend)
np.savetxt(str(file_path,'/common_friends.txt'), common_friend)
np.savetxt(str(file_path,'/preferential.txt'), prefs)
np.savetxt(str(file_path,'/jaccards.txt'), jacard_coefs)


In [ ]:
G = G_Whole
file_path = 'crawled1_whole'
features()
np.savetxt(str(file_path,'/opposite_friends.txt'), opposite_friend)
np.savetxt(str(file_path,'/total_friends.txt'), total_friend)
np.savetxt(str(file_path,'/transistive_friends.txt'), trans_friend)
np.savetxt(str(file_path,'/common_friends.txt'), common_friend)
np.savetxt(str(file_path,'/preferential.txt'), prefs)
np.savetxt(str(file_path,'/jaccards.txt'), jacard_coefs)



In [ ]:
G = G_limited
file_path = 'crawled1_limited'
degree_features()
np.savetxt(str(file_path,'/degree_sources.txt'), degree_sources[:,1])
np.savetxt(str(file_path,'/degree_source_in.txt'), degree_source_in[:,1])
np.savetxt(str(file_path,'/degree_source_out.txt'), degree_source_out[:,1])
np.savetxt(str(file_path,'/degree_target.txt'), degree_target[:,1])
np.savetxt(str(file_path,'/degree_target_in.txt'), degree_target_in[:,1])
np.savetxt(str(file_path,'/degree_target_out.txt'), degree_target_out[:,1])





In [ ]:
G = G_Whole
file_path = 'crawled1_whole'
degree_features()

np.savetxt(str(file_path,'/degree_sources.txt'), degree_sources[:,1])
np.savetxt(str(file_path,'/degree_source_in.txt'), degree_source_in[:,1])
np.savetxt(str(file_path,'/degree_source_out.txt'), degree_source_out[:,1])
np.savetxt(str(file_path,'/degree_target.txt'), degree_target[:,1])
np.savetxt(str(file_path,'/degree_target_in.txt'), degree_target_in[:,1])
np.savetxt(str(file_path,'/degree_target_out.txt'), degree_target_out[:,1])



In [116]:
len(whole_data)

3567526